# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
#os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
SEQUENCE=7
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_0'%SEQUENCE
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_1'%SEQUENCE

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 4.49 ms, sys: 1.17 ms, total: 5.66 ms
Wall time: 4.91 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [ ]:
seq = Sequence()
for f in ProgressBar()(frame[:]):
    seq.add_frame(f, run_ba=False)

  0% (2 of 1101) |                        | Elapsed Time: 0:00:00 ETA:  0:01:18

connected mappoints  179


  0% (10 of 1101) |                       | Elapsed Time: 0:00:01 ETA:  0:02:29

aap
connected mappoints  35


  0% (11 of 1101) |                       | Elapsed Time: 0:00:01 ETA:  0:03:15

...

invalid speed keyframe 843 frame 844 speed 1.0305316030313312
 [[  9.99993347e-01  -3.57463206e-03   7.25945066e-04   1.69596307e-02]
 [  3.57476603e-03   9.99993594e-01  -1.83329613e-04   3.99477585e-02]
 [ -7.25285080e-04   1.85923477e-04   9.99999720e-01  -1.08706136e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
noot True False 83


 76% (847 of 1101) |################      | Elapsed Time: 0:21:51 ETA:  0:07:26

aap
connected mappoints  31


 77% (848 of 1101) |################      | Elapsed Time: 0:21:54 ETA:  0:08:22

invalid speed keyframe 846 frame 847 speed 2.11312114301322
 [[  9.99999383e-01  -1.03351252e-03   4.06922909e-04   1.73229951e-02]
 [  1.03299760e-03   9.99998668e-01   1.26357548e-03   2.45988266e-02]
 [ -4.08228289e-04  -1.26315435e-03   9.99999119e-01  -1.09461246e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
noot True False 93


In [ ]:
%%time
SEQUENCE=7
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_0'%SEQUENCE
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_1'%SEQUENCE

kframe = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    kframe.append(left_frame)

In [12]:
kseq = Sequence()
for f in ProgressBar()(kframe[630:650]):
    kseq.add_keyframe(f, run_ba=False)

  5% (1 of 20) |#                         | Elapsed Time: 0:00:01 ETA:  0:00:22

connected mappoints  0


 10% (2 of 20) |##                        | Elapsed Time: 0:00:02 ETA:  0:00:23

connected mappoints  105


 15% (3 of 20) |###                       | Elapsed Time: 0:00:04 ETA:  0:00:25

...

connected mappoints  110
invalid speed keyframe 1761 frame 1762 speed 0.02683957429844161
 [[  9.99994169e-01  -9.53541264e-04   3.27910909e-03  -1.07278257e-02]
 [  9.57382503e-04   9.99998857e-01  -1.17005762e-03   8.50521530e-03]
 [ -3.27798964e-03   1.17319016e-03   9.99993939e-01  -1.84198570e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 95% (19 of 20) |#######################  | Elapsed Time: 0:00:24 ETA:  0:00:01

connected mappoints  114
invalid speed keyframe 1762 frame 1763 speed 0.007656109467807426
 [[  9.99991016e-01  -2.37334563e-03   3.51203032e-03  -1.51545556e-02]
 [  2.37626633e-03   9.99996834e-01  -8.27688118e-04   1.12846043e-02]
 [ -3.51005481e-03   8.36026201e-04   9.99993490e-01  -1.76542461e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


100% (20 of 20) |#########################| Elapsed Time: 0:00:25 Time: 0:00:25


connected mappoints  111


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [6]:
print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.get_world_pose().flatten() for kf in keyframes ] ])

Keyframes count
 13 



In [ ]:
cv_keyframes = [ seq.keyframes[f] for f in range(len(seq.keyframes)) ]
#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

#cv_keyframes_BA[-1,2:] = cv_keyframes_BA[-1,2:] * 2
#print(cv_keyframes_BA[3])
results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, [5,13]].reshape((-1,2))

In [8]:
cv_keyframes = [ seq.keyframes[f] for f in range(len(seq.keyframes)) ]
#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

#cv_keyframes_BA[-1,2:] = cv_keyframes_BA[-1,2:] * 2
#print(cv_keyframes_BA[3])
results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, [5,13]].reshape((-1,2))

array([[ 0.        ,  0.        ],
       [ 0.05548013, -1.04691944],
       [ 0.03477116, -2.13411294],
       [-0.4324042 , -3.87420913],
       [-0.8218082 , -4.61716318],
       [-1.1909044 , -5.08371176],
       [-1.85038041, -5.74420082],
       [-2.81400045, -6.31998673],
       [-3.1521221 , -6.47106179],
       [-3.52995305, -6.61080743],
       [-4.33370192, -6.85478241],
       [-6.01639211, -7.06477951],
       [-8.10863041, -7.06496957]])

In [15]:
cv_keyframes = [ kseq.keyframes[f] for f in range(0,20) ]
#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

#cv_keyframes_BA[-1,2:] = cv_keyframes_BA[-1,2:] * 2
#print(cv_keyframes_BA[3])
results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, [2]].reshape((-1))

array([ 1.        ,  0.99998669,  0.9999556 ,  0.99991363,  0.99985673,
        0.99980295,  0.99974409,  0.99965332,  0.99962443,  0.99704223,
        0.99142642,  0.99176535,  0.99202048,  0.99224619,  0.99266073,
        0.99306357,  0.99320647,  0.99354051,  0.99392198,  0.99435138])

In [ ]:
len(links) / 50

In [ ]:
#links2 = [ l for l in links if l[1] == 2.0]
#[ l for l in links if int(l[0]) >= len(mappoints)]
#mid2 = [ int(m[0]) for m in mappoints ]
#[ l for l in links2 if int(l[0]) not in mid2 ]
cv_keyframes_BA[:, [5,13]].reshape((-1,2))

In [ ]:

#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
cv_keyframes = [ seq.keyframes[f] for f in range(len(seq.keyframes)) ]

mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

cv_keyframes_np[:,[5,13]]

In [ ]:
countsp = [0] * 50
counts = [0] * 50
for o in outliers:
    if o[2] > 6:
        counts[int(o[0])] += 1
for o in links:
    countsp[int(o[1])] += 1
for i in range(len(counts)):
    counts[i] /= countsp[i] + 0.001
counts

In [ ]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 5.8 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
#m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
#m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for f in cv_keyframes if f.keyframeid in keptframes]
fixed_keyframes1 = [ f for f in f_keyframes if f.keyframeid in keptframes]
mappoints1 = [ f for f in mappoints if int(f[0]) in keptpoints]
links1 = [ (l[0], l[1], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [ ]:
len(mappoints), len(links)

In [ ]:
len(inliers) / len(links)

In [ ]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [ ]:
cv_keyframes1_BA[:,2:]

In [ ]:

cv_keyframes = { seq.keyframes[i] for i in range(0,7,2) }
mappoints = get_mappoints(cv_keyframes)
f_keyframes = { }

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

# print('mappoints count  \n', (len(mappoints[:]), '\n') )
# print('mappoints \n', (mappoints[:]), '\n')
# print('links count  \n', (len(links[:]), '\n') )
# print('links \n', (links[:]), '\n')

# print('Keyframes \n', (cv_keyframes), '\n')
# print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
# print('mappoints len \n', len(mappoints_np), '\n')
# print('Fixed Keyframes \n',f_keyframes[:], '\n')

# print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
# print(len(cv_keyframes_BA), '\n')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

# print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
# print(len(cv_keyframes_BA), '\n')

In [ ]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 10 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for i, f in enumerate(cv_keyframes) if i in keptframes]
fixed_keyframes1 = [ f for i, f in enumerate(f_keyframes) if i + len(cv_keyframes) in keptframes]
mappoints1 = [ f for i, f in enumerate(mappoints) if i in keptpoints]
links1 = [ (m_keptpoints[l[0]], m_keptframes[l[1]], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [ ]:
show(draw_frame(frame[6]))
show(draw_frame(kframe[6]))
obs = [o for o in frame[6].get_observations() if o.cx > 1100]
[ o.get_depth() for o in obs ]

In [ ]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [ ]:
cv_keyframes1_BA[:, 2:]